## **Deepfake Detection with a Convolutional Neural Network**
Daniel Radunsky, Adam Torres Encarnacion

#### **Import Packages**

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam
import pandas as pd

In [7]:
df = pd.read_json(r"C:\Users\kingd\Downloads\deepfake-detection-challenge\train_sample_videos\metadata.json")
df = df.T

In [10]:
df.index.name = "name"
df.head()

,label,split,original
name,,,
aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
abarnvbtwb.mp4,REAL,train,None
abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4


#### **Architecture**

In [ ]:
class Detector(nn.Module):
    def __init__(self):
        super(Detector, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, 1),
            nn.ReLU(),
            nn.Conv2d(16, 4, 1, 0),
            nn.ReLU(),
            nn.BatchNorm2d(4),
            nn.MaxPool2d(8, 8, 0)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(4 * (256 / 8) * (256 / 8), 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

SyntaxError: incomplete input (623605766.py, line 9)

#### **GPU Optimization**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Detector().to(device)

#### **Training Loop**

In [ ]:
loss_function = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

for epoch in range(1000):
    pass